In [2]:
#Importing Necessary Libraries.

from PIL import Image
import numpy as np
import pandas as pd 
import os
import re
import cv2
from pathlib import Path
from scipy.stats import spearmanr

import matplotlib.pyplot as P
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4024)])
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

In [3]:
from scipy.stats import spearmanr as spr

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

import glob2


tf.__version__

'2.2.0'

In [4]:
INPUT_TRUE = "/home/maia/Documents/Universidade/Universidade/UNIVERSIDADE/MESTRADO/CADEIRAS/FPCC2/CIFAR10/OUT/EVAL_EXPLAIN_TRUE/*/*.png"
INPUT_FILES_TRUE = glob2.glob(INPUT_TRUE)

INPUT_RANDOM = "/home/maia/Documents/Universidade/Universidade/UNIVERSIDADE/MESTRADO/CADEIRAS/FPCC2/CIFAR10/OUT/EVAL_EXPLAIN_RANDOM/*/*.png"
INPUT_FILES_RANDOM = glob2.glob(INPUT_RANDOM)


In [5]:
def image_generator(directory, res):
    test_datagen = ImageDataGenerator()
    image_generator = test_datagen.flow_from_directory(
            directory,
            target_size=(res, res),
            batch_size=1,
            shuffle=False,
            class_mode='categorical')
    return image_generator

In [11]:
result_dict = {
    "im_true" : [],
    "im_random" : [],
    "corr" : [],
    "puro_spearman":[],
    "pvalue" : [],
    "method" : []
}

result_dict = {
    "puro_spearman":[],
    "pvalue" : [],
    "method" : []
}


for im_TRUE, im_RANDOM in zip(INPUT_FILES_TRUE, INPUT_FILES_RANDOM):
    im_TRUELOAD = cv2.imread(im_TRUE)
    im_RANDOMLOAD = cv2.imread(im_RANDOM)
    
    puro_spearman = spr(im_TRUELOAD.flatten(), im_RANDOMLOAD.flatten())
    
    #hog = cv2.HOGDescriptor()
    #h_true = hog.compute(im_TRUELOAD)
    #h_random = hog.compute(im_RANDOMLOAD)
    #res_cor = spr(h_true, h_random)
    result_dict["method"].append(im_TRUE.split("/")[13])
    #result_dict["im_true"].append(im_TRUE)
    #result_dict["im_random"].append(im_RANDOM)
    result_dict["puro_spearman"].append(puro_spearman.correlation)
    result_dict["pvalue"].append(puro_spearman.pvalue)

result_df = pd.DataFrame(result_dict)
result_df.to_csv("result.csv")

In [12]:
result_df

,puro_spearman,pvalue,method
0,-0.137996,1.564153e-14,XRAI
1,0.023637,1.902912e-01,XRAI
2,0.036343,4.398899e-02,XRAI
3,0.240561,1.084148e-41,XRAI
4,0.220077,5.208287e-35,XRAI
...,...,...,...
3495,0.117160,7.343167e-11,Blur
3496,0.013206,4.643726e-01,Blur
3497,0.301224,1.850630e-65,Blur
3498,0.037387,3.826060e-02,Blur
